In [1]:
import librosa
import librosa.display as librosa_display
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import os
import time

In [2]:
def prepare_SAVEE_DS(path_audios):
    wav_paths, emotions, actors = [], [], []
    
    for path in tqdm(Path(path_audios).glob("*/*.wav")):
        actor = str(path).split('\\')[-2]
        
        emotion = None
        emo = path.stem
        if emo[0]=='a':
            emotion = 0
        elif emo[0]=='d':
            emotion = 1
        elif emo[0]=='f':
            emotion = 2
        elif emo[0]=='h':
            emotion = 3
        elif emo[0]=='n':
            emotion = 4
        elif emo[:2]=='sa':
            emotion = 5
        elif emo[:2]=='su':
            emotion = 6
            
        wav_paths.append(path)
        emotions.append(emotion)
        actors.append(actor)
    
    return wav_paths, emotions, actors
        

In [3]:
wav_paths, emotions, actors = prepare_SAVEE_DS('SAVEE')

480it [00:00, 120396.24it/s]


In [4]:
mean_signal_length = 100000

def get_feature(paths:str, mfcc_len:int=39, flatten:bool=False):
    features = []
    
    for i, path, in tqdm(enumerate(paths), desc='get features.....'):
        signal, fs = librosa.load(path)
        s_len = len(signal)
        
        if s_len < mean_signal_length:
            pad_len = mean_signal_length - s_len
            pad_rem = pad_len % 2
            pad_len //= 2
            signal = np.pad(signal, (pad_len, pad_len+pad_rem), 'constant', constant_values=0)
            
        else:
            pad_len = s_len - mean_signal_length
            pad_len //= 2
            signal = signal[pad_len:pad_len + mean_signal_length]
            
        mfcc = librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=39)
        mfcc = mfcc.T
        
        features.append(mfcc)
    
    return features

In [5]:
features = get_feature(wav_paths)

get features.....: 480it [00:46, 10.39it/s]


In [6]:
features[0].shape

(196, 39)

In [7]:
X = np.array(features)
y = np.array(emotions)

In [8]:
print(X.shape, y.shape)

os.makedirs('TIMNET-dataset', exist_ok=True)
with open('TIMNET-dataset/SAVEE.npy', 'wb') as f:
    np.save(f, X)
    np.save(f, y)

(480, 196, 39) (480,)
